# Homework 2

This Notebook will detail Homework 2, which involves a basic capacity expansion model formulation described in [Notebook 3](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks)

First, load (or install if necessary) a set of packages you'll need for this assignment...

In [1]:
# Uncomment and run this first line if you need to install or update packages
#import Pkg; Pkg.add("JuMP"); Pkg.add("HiGHS"); Pkg.add("DataFrames"); Pkg.add("CSV")
using JuMP
using HiGHS
using DataFrames
using CSV

### Question 1 - Build the basic thermal generation expansion model

Using the example model in [Notebook 3](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks) as your guide, input the code to create a basic thermal generator capacity expansion model, including [downloading the data for Notebook 3 here](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks/expansion_data) and loading the appropriate csv files.

In [2]:
generators = DataFrame(CSV.File("C:/Users/44780/Documents/GitHub/PowerSystemOptimization/power-systems-optimization/Notebooks/expansion_data/generators_for_expansion.csv"))
demand = DataFrame(CSV.File("C:/Users/44780/Documents/GitHub/PowerSystemOptimization/power-systems-optimization/Notebooks/expansion_data/demand_for_expansion.csv"))

Row,Hour,Demand
,Int64,Int64
1,1,2274
2,2,2581
3,3,2576
4,4,2482
5,5,2396
6,6,2193
7,7,1929
8,8,1715
9,9,1555


In [3]:
NSECost = 9000
# The set of generators from the generators DataFrame
G = generators.G[1:(size(generators,1)-2)]  # note we truncate wind and solar for now
# The set of hours in the demand DataFrame
H = demand.Hour;
Expansion_Model = Model(HiGHS.Optimizer) # using the HiGHS open source solver
@variables(Expansion_Model, begin
        CAP[g in G] >=0          # Generating capacity built (MW)
        GEN[g in G, h in H] >= 0 # Generation in each hour (MWh)
        NSE[h in H] >= 0         # Non-served energy in each hour (MWh)
end)
@constraints(Expansion_Model, begin
    cDemandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == demand.Demand[h]
    cCapacity[g in G, h in H], GEN[g,h] <= CAP[g]
end)
# Note: the objective is now too long to print. It is therefore doubly important to check your code.
@objective(Expansion_Model, Min,
    sum(generators[generators.G.==g,:FixedCost][1]*CAP[g] + 
        sum(generators[generators.G.==g,:VarCost][1]*GEN[g,h] for h in H)
    for g in G) + 
    sum(NSECost*NSE[h] for h in H) 
);
# Note: the objective is now too long to print. It is therefore doubly important to check your code.
@objective(Expansion_Model, Min,
    sum(generators[generators.G.==g,:FixedCost][1]*CAP[g] + 
        sum(generators[generators.G.==g,:VarCost][1]*GEN[g,h] for h in H)
    for g in G) + 
    sum(NSECost*NSE[h] for h in H) 
);

## Question 2: Analytical solution

**A.** Using the data provided above, sort the demand data from highest to lowest hours to create a load duration curve and save this as a vector/array/DataFrame of your choice.

In [20]:
sort!(demand, rev = true)


Row,Hour,Demand
,Int64,Int64
1,8760,2200
2,8759,2195
3,8758,2538
4,8757,2646
5,8756,2708
6,8755,2677
7,8754,2707
8,8753,2616
9,8752,2527


**B.** Now using the cost data provided in '/generators_for_expansion.csv' and the load duration curve above, use the formulas provided in Lecture to determine an analytical solution to the optimal thermal generation expansion decisions (e.g. solve it algebraically rather than use an optimization solver to find the solution). 

Report the optimal capacity of each generation source and compare to the solution from the optimization model above. 

Show your work in cells below, using Julia to perform calculations. Explain your steps using inline code comments (e.g. `# Comment`) or by interspersing Markdown cells.  

Tip: round your solutions for the crossover hour between each technology to the nearest integer (as we have discrete hours in the time series).

**C.** Now change the fuel cost of natural gas to \$8.00/MMBtu, recalculate the variable cost of CCGTs and CTs, and solve again for the optimal generation capacity mix. Describe what changes in your capacity results and what doesn't, and provide an explanation.

## Question 3 - Expansion with renewables

**A.** Using JuMP/Julia, implement an optimization model based on the formulation for optimal thermal+renewable capacity expansion provided in Section 2 of [Notebook 3](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks). 

**B.** Solve the model to determine the optimal capacity when wind and solar are available resources and extract results for generation and capacity.

**C.** What happens to the total firm generation and maximum MW of non-served energy? What does this imply about the capacity value of solar and/or wind built in the optimal capacity mix?

## Question 4: Brownfield Expansion Model

**A.** Now implement an optimization model based on the formulation for optimal "brownfield" thermal+renewable capacity expansion (e.g. with existing generators) provided in Section 3 of [Notebook 3](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks).

Use the following data for fixed and variable costs of existing gas capacity. Note: unlike in the formulation in Notebook 3, there is no existing renewable capacity here to consider (only thermal).

In [12]:
# Load new generator options
#path = joinpath([REPLACE THIS WITH PATH TO YOUR power-systems-optimization DIRECTORY HERE],"Notebooks","expansion_data")
path = joinpath("/Users/jdj2/Documents/GitHub/power-systems-optimization","Notebooks","expansion_data")
generators = DataFrame(CSV.File(joinpath(path,"generators_for_expansion.csv")))
# Add parameters for existing CCGTs, with the set index "Old"
push!(generators, ["Old_CC" "Existing CCGT" 0 40000 5 7.5 4 0 0 0 0 40000 30])
# Add parameters for existing CTs, with the set index "Old"
push!(generators, ["Old_CT" "Existing CT" 0 30000 11 11.0 4 0 0 0 0 30000 55])

# Set installed capacity for existing CCGTs:
ExistingCap_CCGT = 1260 # Approximate actual existing capacity in SDGE
ExistingCap_CT = 925 # Approximate actual existing capacity in SDGE
# Add new column to generators Data Frame
generators[!,:ExistingCap] = [0,0,0,0,0,0, ExistingCap_CCGT, ExistingCap_CT];

**B.** Solve the model to determine the optimal capacity when with existing generators and extract results for generation and capacity (including retirements).